# Choquet Classifier Implementation
## Replication of Figure 3(a) from "Nonlinear Classification by Genetic Algorithm with Signed Fuzzy Measure"

**Objective**: Implement a Genetic Algorithm to find the optimal Choquet hyperplane for binary classification in 2D space and replicate Figure 3(a) from the paper.

**Key Components**:
1. **Data Generation**: Create 200 synthetic 2D points with known class labels
2. **Choquet Integral**: Implement 2D Choquet integral calculation
3. **Genetic Algorithm**: Optimize classifier parameters using GA
4. **Visualization**: Plot results and decision boundary

---


In [37]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import random
from typing import List, Tuple, Dict
import json
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")


Libraries imported successfully!
NumPy version: 2.3.3
Matplotlib version: 3.10.7


## Phase 1: Understanding the Problem

### The Choquet Classifier
The paper uses a **Choquet integral** to create a nonlinear decision boundary. In 2D space, we have:

- **3 measure values**: μ₁, μ₂, μ₁₂ (for features x₁, x₂, and their interaction)
- **Classifier equation**: (C)∫ (a + b·f)dμ - B = 0
- **Fitness function**: Penalized total signed Choquet distance D_c

### Key Equations
1. **Choquet Integral (2D case)**:
   - If h₁ ≤ h₂: C = h₁ × μ₁₂ + (h₂ - h₁) × μ₂
   - If h₂ < h₁: C = h₂ × μ₁₂ + (h₁ - h₂) × μ₁
   - Where hᵢ = aᵢ + bᵢ × xᵢ

2. **Fitness Function**: Maximize D_c with penalty factor 111 for misclassifications


In [38]:
# Complete ChoquetClassifier class with all methods
class ChoquetClassifier:
    """
    Implements the Choquet integral-based classifier with Genetic Algorithm optimization.
    """
    
    def __init__(self, population_size: int = 100, penalty_c: int = 111, 
                 bits_per_param: int = 10, mutation_rate: float = 0.01):
        self.population_size = population_size
        self.penalty_c = penalty_c
        self.bits_per_param = bits_per_param
        self.mutation_rate = mutation_rate
        self.chromosome_length = 8 * bits_per_param
        
        # Parameters from paper's Table III
        self.paper_params = {
            'mu_1': 0.1802, 'mu_2': 0.5460, 'mu_12': 0.1389,
            'B': 0.0917, 'a': [0.0, 0.0], 'b': [1.0, 1.0]
        }
        print(f"ChoquetClassifier initialized with population size {self.population_size}")
    
    def choquet_integral_2d(self, x, a, b, mu):
        """Calculate 2D Choquet integral"""
        mu_1, mu_2, mu_12 = mu
        h = a + b * x
        h1, h2 = h[0], h[1]
        
        if h1 <= h2:
            return h1 * mu_12 + (h2 - h1) * mu_2
        else:
            return h2 * mu_12 + (h1 - h2) * mu_1
    
    def generate_data(self, n_samples=200):
        """Generate synthetic data using paper's parameters"""
        print("Generating synthetic data...")
        data = np.random.uniform(0, 1, (n_samples, 2))
        
        mu_1, mu_2, mu_12 = self.paper_params['mu_1'], self.paper_params['mu_2'], self.paper_params['mu_12']
        B = self.paper_params['B']
        a, b = np.array(self.paper_params['a']), np.array(self.paper_params['b'])
        
        labels = []
        for point in data:
            choquet_value = self.choquet_integral_2d(point, a, b, [mu_1, mu_2, mu_12])
            labels.append(1 if choquet_value >= B else -1)
        
        labels = np.array(labels)
        print(f"Generated {n_samples} samples: {np.sum(labels==1)} Class A, {np.sum(labels==-1)} Class A'")
        return data, labels
    
    def decode_chromosome(self, chromosome):
        """Decode binary chromosome to parameters"""
        params = {}
        for i in range(8):
            start_idx = i * self.bits_per_param
            end_idx = start_idx + self.bits_per_param
            binary_segment = chromosome[start_idx:end_idx]
            decimal_value = int(binary_segment, 2) / (2**self.bits_per_param - 1)
            
            if i in [0, 1, 2, 3]:  # mu_1, mu_2, mu_12, B in [0, 1]
                param_value = decimal_value
            else:  # a0, a1, b0, b1 in [-1, 1]
                param_value = 2 * (decimal_value - 0.5)
            
            param_names = ['mu_1', 'mu_2', 'mu_12', 'B', 'a0', 'a1', 'b0', 'b1']
            params[param_names[i]] = param_value
        return params
    
    def fitness_function(self, chromosome, data, labels):
        """Calculate fitness with penalty for misclassifications"""
        params = self.decode_chromosome(chromosome)
        mu_1, mu_2, mu_12 = params['mu_1'], params['mu_2'], params['mu_12']
        B = params['B']
        a = np.array([params['a0'], params['a1']])
        b = np.array([params['b0'], params['b1']])
        
        # Ensure valid ranges
        mu_1, mu_2, mu_12 = max(0, min(1, mu_1)), max(0, min(1, mu_2)), max(0, min(1, mu_12))
        mu_12 = max(mu_12, max(mu_1, mu_2))
        
        total_distance = 0.0
        correct = 0
        
        for i, point in enumerate(data):
            choquet_value = self.choquet_integral_2d(point, a, b, [mu_1, mu_2, mu_12])
            signed_distance = choquet_value - B
            
            if labels[i] == 1:  # Class A
                if choquet_value < B:  # Misclassified
                    signed_distance *= self.penalty_c
                else:
                    correct += 1
            else:  # Class A'
                if choquet_value >= B:  # Misclassified
                    signed_distance *= self.penalty_c
                else:
                    correct += 1
            
            total_distance += signed_distance
        
        return total_distance + correct * 1000
    
    def create_random_chromosome(self):
        """Create random binary chromosome"""
        return ''.join(random.choice('01') for _ in range(self.chromosome_length))
    
    def roulette_wheel_selection(self, population, fitness_scores):
        """Select parent using roulette wheel"""
        total_fitness = sum(fitness_scores)
        if total_fitness == 0:
            return random.choice(population)
        
        r = random.random()
        cumulative = 0.0
        for i, fitness in enumerate(fitness_scores):
            cumulative += fitness / total_fitness
            if r <= cumulative:
                return population[i]
        return population[-1]
    
    def crossover(self, parent1, parent2):
        """Single-point crossover"""
        point = random.randint(1, self.chromosome_length - 1)
        offspring1 = parent1[:point] + parent2[point:]
        offspring2 = parent2[:point] + parent1[point:]
        return offspring1, offspring2
    
    def mutate(self, chromosome):
        """Apply mutation"""
        chromosome_list = list(chromosome)
        for i in range(len(chromosome_list)):
            if random.random() < self.mutation_rate:
                chromosome_list[i] = '1' if chromosome_list[i] == '0' else '0'
        return ''.join(chromosome_list)
    
    def run_genetic_algorithm(self, data, labels, max_generations=100):
        """Run the complete genetic algorithm"""
        print("Starting Genetic Algorithm...")
        population = [self.create_random_chromosome() for _ in range(self.population_size)]
        
        best_fitness_history = []
        no_improvement = 0
        
        for generation in range(max_generations):
            # Evaluate fitness
            fitness_scores = [self.fitness_function(chrom, data, labels) for chrom in population]
            
            # Find best
            best_idx = np.argmax(fitness_scores)
            best_fitness = fitness_scores[best_idx]
            best_chromosome = population[best_idx]
            best_params = self.decode_chromosome(best_chromosome)
            
            best_fitness_history.append(best_fitness)
            
            if generation % 10 == 0:
                print(f"Generation {generation}: Best fitness = {best_fitness:.2f}")
            
            # Check stopping condition
            if generation > 0 and best_fitness <= best_fitness_history[-2]:
                no_improvement += 1
            else:
                no_improvement = 0
            
            if no_improvement >= 10:
                print(f"Stopping early at generation {generation}")
                break
            
            # Create new population
            new_population = []
            
            # Keep best half (elitism)
            sorted_indices = np.argsort(fitness_scores)[::-1]
            elite_size = self.population_size // 2
            for i in range(elite_size):
                new_population.append(population[sorted_indices[i]])
            
            # Generate offspring
            while len(new_population) < self.population_size:
                parent1 = self.roulette_wheel_selection(population, fitness_scores)
                parent2 = self.roulette_wheel_selection(population, fitness_scores)
                offspring1, offspring2 = self.crossover(parent1, parent2)
                offspring1 = self.mutate(offspring1)
                offspring2 = self.mutate(offspring2)
                new_population.extend([offspring1, offspring2])
            
            population = new_population[:self.population_size]
        
        print(f"GA completed after {generation + 1} generations")
        return {
            'best_params': best_params,
            'best_fitness': best_fitness,
            'fitness_history': best_fitness_history,
            'generations': generation + 1
        }
    
    def plot_results(self, data, labels, best_params):
        """Plot classification results and decision boundary"""
        plt.figure(figsize=(10, 8))
        
        # Plot data points
        class_a_mask = labels == 1
        class_a_prime_mask = labels == -1
        
        plt.scatter(data[class_a_mask, 0], data[class_a_mask, 1], 
                   c='red', marker='o', label='Class A', s=50, alpha=0.7)
        plt.scatter(data[class_a_prime_mask, 0], data[class_a_prime_mask, 1], 
                   c='blue', marker='s', label="Class A'", s=50, alpha=0.7)
        
        # Create decision boundary
        xx, yy = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))
        grid_points = np.c_[xx.ravel(), yy.ravel()]
        
        mu_1, mu_2, mu_12 = best_params['mu_1'], best_params['mu_2'], best_params['mu_12']
        B = best_params['B']
        a = np.array([best_params['a0'], best_params['a1']])
        b = np.array([best_params['b0'], best_params['b1']])
        
        decision_values = []
        for point in grid_points:
            choquet_value = self.choquet_integral_2d(point, a, b, [mu_1, mu_2, mu_12])
            decision_values.append(choquet_value - B)
        
        decision_values = np.array(decision_values).reshape(xx.shape)
        plt.contour(xx, yy, decision_values, levels=[0], colors='black', linewidths=2, linestyles='--')
        
        plt.xlabel('Feature x1')
        plt.ylabel('Feature x2')
        plt.title('Replication of Figure 3(a): Choquet Classifier Results')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
    
    def calculate_accuracy(self, data, labels, params):
        """Calculate classification accuracy"""
        correct = 0
        mu_1, mu_2, mu_12 = params['mu_1'], params['mu_2'], params['mu_12']
        B = params['B']
        a = np.array([params['a0'], params['a1']])
        b = np.array([params['b0'], params['b1']])
        
        for i, point in enumerate(data):
            choquet_value = self.choquet_integral_2d(point, a, b, [mu_1, mu_2, mu_12])
            predicted_label = 1 if choquet_value >= B else -1
            if predicted_label == labels[i]:
                correct += 1
        
        return correct / len(data)

print("ChoquetClassifier class defined successfully!")


ChoquetClassifier class defined successfully!


## Phase 2: Data Generation

### Function 1: generate_data()
**Purpose**: Create the 200 sample points used in Figure 3(a)

**Method**: 
1. Generate 200 random points (x₁, x₂) in [0,1] × [0,1]
2. Use the paper's final parameters to determine class labels
3. Apply the Choquet integral: if C ≥ B, label as Class A (155 points), otherwise Class A' (45 points)


In [39]:
    def generate_data(self, n_samples: int = 200) -> Tuple[np.ndarray, np.ndarray]:
        """
        Generate synthetic 2D classification data using the paper's final parameters.
        
        This function reverse-engineers the data generation process by using the 
        paper's final parameters to create a dataset that should be separable
        by the Choquet classifier.
        
        Args:
            n_samples: Number of samples to generate (default: 200)
            
        Returns:
            Tuple of (data_points, labels) where labels are 1 for Class A, -1 for Class A'
        """
        print("=" * 50)
        print("GENERATING SYNTHETIC DATA")
        print("=" * 50)
        print("Using paper's final parameters to create training data...")
        
        # Generate random points in [0, 1] x [0, 1]
        data = np.random.uniform(0, 1, (n_samples, 2))
        print(f"Generated {n_samples} random points in [0,1] x [0,1]")
        
        # Use paper's final parameters to determine class labels
        mu_1, mu_2, mu_12 = self.paper_params['mu_1'], self.paper_params['mu_2'], self.paper_params['mu_12']
        B = self.paper_params['B']
        a = np.array(self.paper_params['a'])
        b = np.array(self.paper_params['b'])
        
        print(f"Using parameters: μ₁={mu_1:.4f}, μ₂={mu_2:.4f}, μ₁₂={mu_12:.4f}, B={B:.4f}")
        print(f"Vectors: a={a}, b={b}")
        
        labels = []
        choquet_values = []
        
        for i, point in enumerate(data):
            # Calculate Choquet integral for this point
            choquet_value = self.choquet_integral_2d(point, a, b, [mu_1, mu_2, mu_12])
            choquet_values.append(choquet_value)
            
            # Assign class label based on decision boundary
            if choquet_value >= B:
                labels.append(1)  # Class A
            else:
                labels.append(-1)  # Class A'
        
        labels = np.array(labels)
        choquet_values = np.array(choquet_values)
        
        # Verify we get approximately the right distribution
        class_a_count = np.sum(labels == 1)
        class_a_prime_count = np.sum(labels == -1)
        
        print(f"\nData generation results:")
        print(f"  Class A: {class_a_count} samples ({class_a_count/n_samples*100:.1f}%)")
        print(f"  Class A': {class_a_prime_count} samples ({class_a_prime_count/n_samples*100:.1f}%)")
        print(f"  Expected: ~155 Class A, ~45 Class A'")
        
        # Show some statistics
        print(f"\nChoquet integral statistics:")
        print(f"  Min value: {choquet_values.min():.4f}")
        print(f"  Max value: {choquet_values.max():.4f}")
        print(f"  Mean value: {choquet_values.mean():.4f}")
        print(f"  Decision threshold B: {B:.4f}")
        
        return data, labels


## Phase 3: Core Logic Implementation

### Function 2: choquet_integral_2d()
**Purpose**: Calculate the Choquet integral for a single 2D data point

**Mathematical Foundation**:
- Compute integrand: hᵢ = aᵢ + bᵢ × xᵢ for i=1,2
- Sort h values and apply Choquet integral formula
- In 2D: If h₁ ≤ h₂: C = h₁ × μ₁₂ + (h₂ - h₁) × μ₂
- If h₂ < h₁: C = h₂ × μ₁₂ + (h₁ - h₂) × μ₁


In [40]:
def choquet_integral_2d(self, x: np.ndarray, a: np.ndarray, b: np.ndarray, 
                           mu: List[float]) -> float:
        """
        Calculate the Choquet integral for a 2D data point.
        
        This is the core mathematical function that computes the Choquet integral
        for a single 2D point. The Choquet integral is a generalization of the
        Lebesgue integral that can handle non-additive measures.
        
        Args:
            x: 2D data point [x1, x2]
            a: Vector a = [a1, a2] 
            b: Vector b = [b1, b2]
            mu: Measure values [mu_1, mu_2, mu_12]
            
        Returns:
            Choquet integral value
        """
        mu_1, mu_2, mu_12 = mu
        
        # Step 1: Compute integrand h_i = a_i + b_i * x_i
        h = a + b * x
        h1, h2 = h[0], h[1]
        
        # Step 2: Apply Choquet integral formula based on ordering
        if h1 <= h2:
            # Case 1: h1 <= h2
            # C = h1 * mu_12 + (h2 - h1) * mu_2
            choquet_value = h1 * mu_12 + (h2 - h1) * mu_2
        else:
            # Case 2: h2 < h1  
            # C = h2 * mu_12 + (h1 - h2) * mu_1
            choquet_value = h2 * mu_12 + (h1 - h2) * mu_1
            
        return choquet_value
